1. 原始檔取得
2. 從JFULL提段落、句子
3. AI標註段落、句子，最後prediction檔案，opinion, fee, sub
4. 將prediction過濾，只留下"目標"，opinion, fee, sub
5. 將含有目標的prediction，其JID對應回去原始檔案，只保留有關的判決書，建立basic_df
6. 將basic_df中的jud_full，透過re抽出，'case_num', 'basic_info', 'case_type', 'court_type', 'jud_date', 'jud_url', 'syllabus'

# Funcs

In [2]:
import pandas as pd
import os
import re
from tqdm import tqdm

def jud_read_csv(path, lineterminator=''):
    # 讀取檔案並顯示基本資訊
    print('---Start reading csv file---')
    if type(path)==str:
        print(path)
        if lineterminator=="":
            df = pd.read_csv(path)
        else:
            df = pd.read_csv(path, lineterminator=lineterminator)
        print(os.path.basename(path))
    else:
        df = path
    print('Length:', len(df), '\n')
    print(df.iloc[-1])
    print('---Finish reading csv file---\n')

    return df

# 讀取檔案

In [2]:
import pandas as pd
import os
import re
from tqdm import tqdm

def jud_read_csv(path, lineterminator=''):
    # 讀取檔案並顯示基本資訊
    print('---Start reading csv file---')
    print(path)
    if lineterminator=="":
        df = pd.read_csv(path)
    else:
        df = pd.read_csv(path, lineterminator=lineterminator)
    print(os.path.basename(path))
    print('Length:', len(df), '\n')
    print(df.iloc[-1])
    print('---Finish reading csv file---\n')

    return df
# fee_path = '/workspace/data/CDB-2017-2021/3. AI annotated predictions/2017_2021判決書_無簡_paragraph_20240331_predicted_fee.csv'
# sub_path = '/workspace/data/CDB-2017-2021/3. AI annotated predictions/2017_2021判決書_無簡_paragraph_20240331_predicted_sub.csv'
# opinion_path = '/workspace/data/CDB-2017-2021/3. AI annotated predictions/2017_2021判決書_無簡_sentence_20240331_predicted_opinions.csv'
# fee = jud_read_csv(fee_path, '\n')
# sub = jud_read_csv(sub_path, '\n')
# opinion = jud_read_csv(opinion_path, '\n')
ori_df = jud_read_csv('/workspace/data/CDB-2017-2021/1. original/2017_2021_juds_包含有rn_remove_duplicated.csv')


---Start reading csv file---
/workspace/data/CDB-2017-2021/1. original/2017_2021_juds_包含有rn_remove_duplicated.csv
2017_2021_juds_包含有rn_remove_duplicated.csv
Length: 4132137 

Unnamed: 0.1                                              4135232
Unnamed: 0                                                4135232
JID                                   KSDV,112,司促,2020,20230213,1
JFULL           臺灣高雄地方法院支付命令\r\n112年度司促字第2020號\r\n債  權  人  良京實...
Name: 4132136, dtype: object
---Finish reading csv file---



In [4]:
# print(len(ori_df))
# ori_df = ori_df.dropna()
# print(len(ori_df))
# ori_df = ori_df.drop_duplicates(subset=['JID', 'JFULL'], keep='first')
# print(len(ori_df))
# ori_df.to_csv('/workspace/data/CDB-2017-2021/1. original/2017_2021_juds_包含有rn_remove_duplicated.csv', encoding='utf-8-sig')

# 從prediction移除非目標

In [3]:
def jud_filtered_target(input_path, save_file=False, output_path=""):
    print('---Start remaining target rows---')
    print('Reading csv file...')
    output_df = pd.read_csv(input_path, lineterminator='\n')
    print('Input Length:', len(output_df))
    print(' - Input Last Row -\n', output_df.iloc[-1])
    print(' - ')
    output_df = output_df[output_df['prediction']=='目標']
    # output_df = output_df[~output_df['prediction'].isin(['目標', '無關'])]
    # output_df = output_df[output_df['prediction']=='無關']
    print('Target Length:', len(output_df))
    print(' - Target Last Row - \n', output_df.iloc[-1])
    print(' - ')
    if save_file:
        if output_path=="":
            output_file_name = os.path.basename(input_path)
            output_file_name = output_file_name.split('.')[0] + '_target.csv'
            output_path = os.path.dirname(input_path) + '/' + output_file_name
        print('Saved File: ', output_path)
        output_df.to_csv(output_path, encoding='utf-8-sig', index=False, lineterminator='\n')
        print('Reading saved file...')

        saved_df = pd.read_csv(output_path, lineterminator='\n')
        print('Saved Length:', len(saved_df))
        print(' - Saved Last Row - \n', saved_df.iloc[-1])
        print(' - ')
        
        print('Target Length is equal to Saved Length:', len(output_df)==len(saved_df))
    print('---Finish remaining target rows---\n')
    return output_df

fee_path = '/workspace/data/CDB-2017-2021/3. AI annotated predictions/2017_2021判決書_無簡_paragraph_20240331_predicted_fee.csv'
sub_path = '/workspace/data/CDB-2017-2021/3. AI annotated predictions/2017_2021判決書_無簡_paragraph_20240331_predicted_sub.csv'
opinion_path = '/workspace/data/CDB-2017-2021/3. AI annotated predictions/2017_2021判決書_無簡_sentence_20240331_predicted_opinions.csv'

fee = jud_filtered_target(fee_path, False)
sub = jud_filtered_target(sub_path, False)
opinion = jud_filtered_target(opinion_path, False)

---Start remaining target rows---
Reading csv file...
Input Length: 3638732
 - Input Last Row -
 Unnamed: 0                       3638731
JID           TCDM,111,訴,2018,20230201,1
sentence      本案經檢察官潘曉琪提起公訴，檢察官劉世豪到庭執行職務
prediction                            無關
Name: 3638731, dtype: object
 - 
Target Length: 127513
 - Target Last Row - 
 Unnamed: 0                                              3638711
JID                                 TCDM,111,金訴,2018,20230112,1
sentence      3.再查，被告於警詢、檢察事務官詢問及本院審理時供認：我一開始看到該投資廣告時有覺得不合理，...
prediction                                                   目標
Name: 3638711, dtype: object
 - 
---Finish remaining target rows---

---Start remaining target rows---
Reading csv file...
Input Length: 3638732
 - Input Last Row -
 Unnamed: 0                       3638731
JID           TCDM,111,訴,2018,20230201,1
sentence      本案經檢察官潘曉琪提起公訴，檢察官劉世豪到庭執行職務
prediction                            無關
Name: 3638731, dtype: object
 - 
Target Length: 1775704
 - Target Last Row 

# 過濾basic_info的判決書資料
basic_info中的JID若與prediction類別中的JID一致，則保留basic_info中該筆資料

In [4]:
def jud_filter_basic_info(basic_info_path, prediction_path):
    if type(basic_info_path) == pd.DataFrame and type(prediction_path) == pd.DataFrame:
        basic_df = basic_info_path
        prediction_df = prediction_path
    elif type(prediction_path) != pd.DataFrame and type(basic_info_path) == pd.DataFrame:
        basic_df = basic_info_path
        prediction_df = pd.read_csv(prediction_path, lineterminator='\n')
        print('prediction_df Length:', len(prediction_df))
    else:
        basic_df = pd.read_csv(basic_info_path, lineterminator='\n')
        prediction_df = pd.read_csv(prediction_path, lineterminator='\n')
    
    print('---Start filtering basic_info---')
    print('basic_info Length:',  len(basic_df))
    filtered_df = basic_df[basic_df['JID'].isin(prediction_df['JID'])]
    # filtered_df = basic_df[basic_df['JID'].isin(prediction_df['JID'])].copy()
    # filtered_df['remained'] = True
    print('Filtered basic_info Length:',  len(filtered_df))
    print('-')
    print('Not in basic_info Length:', len(prediction_df[~prediction_df['JID'].isin(basic_df['JID'])]))
    
    # # 直接修改原始basic_df
    basic_df.loc[filtered_df.index, 'remained'] = True
    print('---Finish filtering basic_info---\n')

    return basic_df
    # return filtered_df
fee_target_path = '/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/2017_2021判決書_無簡_paragraph_20240331_predicted_fee_target.csv'
sub_target_path = '/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/2017_2021判決書_無簡_paragraph_20240331_predicted_sub_target.csv'
opinion_target_path = '/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/2017_2021判決書_無簡_sentence_20240331_predicted_opinions_target.csv'
filtered_df = jud_filter_basic_info(ori_df, fee_target_path)
filtered_df = jud_filter_basic_info(ori_df, sub_target_path)
filtered_df = jud_filter_basic_info(ori_df, opinion_target_path)
filtered_df = ori_df[ori_df['remained']==True]
print('Filtered basic_info length:', len(filtered_df))


prediction_df Length: 127513
---Start filtering basic_info---
basic_info Length: 4132137
Filtered basic_info Length: 57295
-
Not in basic_info Length: 0
---Finish filtering basic_info---

prediction_df Length: 1775704
---Start filtering basic_info---
basic_info Length: 4132137
Filtered basic_info Length: 210327
-
Not in basic_info Length: 0
---Finish filtering basic_info---

prediction_df Length: 1156480
---Start filtering basic_info---
basic_info Length: 4132137
Filtered basic_info Length: 176210
-
Not in basic_info Length: 0
---Finish filtering basic_info---

Filtered basic_info length: 211813


In [9]:
print(len(filtered_df.dropna()))
print(len(filtered_df[filtered_df['remained']==True]))
output_csv_path = '/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/20240616_2017_2021_juds_包含有rn_filtered_basic_info.csv'
# filtered_df.to_csv(output_csv_path, encoding='utf-8-sig', index=False)

211813
211813


In [10]:
output_csv_path = '/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/20240616_2017_2021_juds_包含有rn_filtered_basic_info.csv'

filtered_df = jud_read_csv(output_csv_path)

---Start reading csv file---
/workspace/data/CDB-2017-2021/20240616_2017_2021_juds_包含有rn_filtered_basic_info.csv
20240616_2017_2021_juds_包含有rn_filtered_basic_info.csv
Length: 211813 

Unnamed: 0.1                                              4135212
Unnamed: 0                                                4135212
JID                                    TCDM,111,訴,2018,20230201,1
JFULL           臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
remained                                                     True
Name: 211812, dtype: object
---Finish reading csv file---



# RE產生基本資料欄位

In [64]:
# # ['UID', 'JID', 'jud_full', 'case_num', 'basic_info', 'case_type', 'court_type', 'jud_date', 'jud_url', 'syllabus']

# Read basic_info
basic_info_path = '/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/20240616_2017_2021_juds_包含有rn_filtered_basic_info_case_num_type_basic_info.csv'
basic_info_df = jud_read_csv(basic_info_path)


---Start reading csv file---
/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/20240616_2017_2021_juds_包含有rn_filtered_basic_info_case_num_type_basic_info.csv
20240616_2017_2021_juds_包含有rn_filtered_basic_info_case_num_type_basic_info.csv
Length: 211813 

Unnamed: 0                                              4135212
JID                                  TCDM,111,訴,2018,20230201,1
JFULL         臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                            臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info             公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                                        違反槍砲彈藥刀械管制條例案件
Name: 211812, dtype: object
---Finish reading csv file---



## Get case_num, basic_info, case_type

In [23]:
# # # JID
# # 20240305 re jud_full
# Get case_num, basic_info, case_type
def get_re_multi_matched_list(input_text_list, rule_patterns, remove_special_signs=False):
    print('Start get_re_multi_matched_list()')
    error_signs = ['\xa0', '\u3000', '\u30005', '\u3000111', '\u30004', '\u300011', '\u30003', '\u300024', '&nbsp;', '\u3000110', '\u300012', '\u300030', ' ', '\r']
    matched_indices = [[] for _ in range(len(rule_patterns))]
    empty_list = []
    print('matched_indices', matched_indices)
    # ---------------------------
    if remove_special_signs:
        for input_list_order in range(len(input_text_list)):
            input_text = input_text_list[input_list_order][1]
            for sign in error_signs:
                input_text = input_text.replace(sign, '')
            input_text_list[input_list_order][1] = input_text
    # ---------------------------
    for input_list_order in tqdm(range(len(input_text_list))):
        input_text = input_text_list[input_list_order][1]
        JID = input_text_list[input_list_order][0]
        for rule_index, pattern in enumerate(rule_patterns):
            search_matched = re.search(pattern, input_text, re.DOTALL)
            if search_matched:
                # search_matched_groups = search_matched.groups()
                # length_below_50 = True
                # for input_list_order in range(3):
                #     if len(search_matched_groups[input_list_order]) > 50:
                #         length_below_50 = False
                #         break
                # if length_below_50:
                #     matched_indices[rule_index].append([JID, search_matched])
                #     break
                matched_indices[rule_index].append([input_list_order, JID, search_matched.groups(), [len(group) for group in search_matched.groups()]])
                break

        if not search_matched:
            empty_list.append([input_list_order, JID, input_text])
    matched_length = [len(matched) for matched in matched_indices]
    print('matched:', matched_length)
    print('empty:', len(empty_list))
    return [matched_indices, empty_list]


# 第一階段縮減
jud_full_4000_texts = [[row['JID'], row['JFULL'][:4000]] for index, row in basic_info_df.iterrows()]
before_syllabus_list = get_re_multi_matched_list(jud_full_4000_texts, [r'(.+?)主 *?文'], True)
print(before_syllabus_list[1])

before_syllabus_texts = [[JID, re_text[0]] for _, JID, re_text, _ in before_syllabus_list[0][0]]
basic_info_patterns = [
        r'(.+?第\d+號)\n(.+?)上列.+?因(.+?)[（|、|，]',
        r'(.+?號)\n([上訴人|聲請人|公訴人|自訴人].+?)[上|上列|以上|下列].+?因(.+?)[（|、|，]',
        r'(.+?第\d+號)\n?(.+?)[上|上列|以上|下列].+?因(.+?)[（|、|，]',
        r'(.+?)\n([上訴人|聲請人|公訴人|自訴人].+)\n(.+?案件)',
]
case_num_basic_info_case_type_list = get_re_multi_matched_list(before_syllabus_texts, basic_info_patterns, False)


# 第一階段沒有matched
empty_texts = [[JID, input_text] for input_list_index, JID, input_text in before_syllabus_list[1]]
empty_case_num_basic_info_case_type_list = get_re_multi_matched_list(empty_texts, basic_info_patterns, False)
empty_case_num_basic_info_case_type_list[1]


Start get_re_multi_matched_list()
matched_indices [[]]


100%|██████████| 211813/211813 [00:01<00:00, 143450.79it/s]


matched: [211763]
empty: 50
[[189, 'TPSM,106,台上,5,20170125', '最高法院刑事判決一○六年度台上字第五號\n上訴人陳居德\n選任辯護人羅明通律師\n蕭仰歸律師\n上列上訴人因違反證券交易法案件，不服台灣高等法院中華民國\n一０四年三月十日第二審更審判決（一０二年度重金上更㈣字第\n九號，起訴案號：台灣台北地方法院檢察署九十二年度偵字第六\n三０六、一０七八九號），提起上訴，本院判決如下：\n主○\n上訴駁回。\n理由\n按刑事訴訟法第三百七十七條規定：上訴於第三審法院，非以判\n決違背法令為理由，不得為之。是提起第三審上訴，應以原判決\n違背法令為理由，係屬法定要件。如果上訴理由書狀並未依據卷\n內訴訟資料，具體指摘原判決不適用何種法則或如何適用不當，\n或所指摘原判決違法情事，顯與法律規定得為第三審上訴理由之\n違法情形，不相適合時，均應認其上訴為違背法律上之程式，予\n以駁回。\n本件原判決撤銷第一審關於上訴人陳居德部分之科刑判決，改判\n仍依修正前刑法牽連犯之規定，從一重論處上訴人陳居德共同違\n反對於在證券交易所上市之有價證券，不得有意圖抬高集中交易\n市場某種有價證券之交易價格，以他人名義，對該有價證券，連\n續以高價買入行為之規定（下稱連續高價買入）罪刑（牽連犯民\n國七十七年一月二十九日修正公布，自同年月三十一日施行之證\n券交易法《下稱行為時證交法》第一百五十五條第一項第四款之\n連續高價買入罪、同條項第六款之直接或間接從事其他影響集中\n交易市場某種有價證券交易價格之操縱行為《下稱操縱股價行為\n》罪。經適用刑事妥速審判法第七條規定減輕其刑後，處有期徒\n刑一年，減為有期徒刑六月），已詳敘其調查證據之結果及證據\n取捨並認定事實之理由；所為論斷，均有卷存證據資料可資覆按\n。\n上訴人之上訴意旨略以：\n\uf6b0原審踐行之訴訟程序，有下列違法之處：\n㈠依起訴書犯罪事實及證據並所犯法條欄之記載，檢察官並未起\n訴上訴人違反行為時證交法第一百五十五條第一項第六款之規\n定，第一審亦未就此部分為調查、審理及判決。原審於審判期\n日，於踐行告知義務時，亦僅告知「罪名，詳如起訴書、原審\n（指第一審）判決書、最高法院判決書所載」等語，並未告知\n上訴人另涉上開罪名，復未就此部分事實為調查，無異剝

100%|██████████| 211763/211763 [00:01<00:00, 162123.13it/s]


matched: [209204, 2063, 12, 411]
empty: 73
Start get_re_multi_matched_list()
matched_indices [[], [], [], []]


100%|██████████| 50/50 [00:05<00:00,  8.80it/s]

matched: [29, 2, 0, 1]
empty: 18


[[3,
  'CTDV,106,審訴,148,20170222,1',
  '臺灣橋頭地方法院民事裁定106年度審訴字第148號\n原告吳品緯\n被告陳信嘉\n兩造間依本院105年度交簡字第2362號刑事判決附帶提起民事損\n害賠償事件，原告請求之眼鏡、機車修理費用新臺幣（下同）90\n0元、13,250元（合計14,150元），非屬上開刑事判決附帶民事\n訴訟所得請求範圍內，原告應補繳第一審裁判費1,000元。茲依\n民事訴訟法第249條第1項但書之規定，限原告於收受本裁定送\n達5日內補繳，逾期不繳，即駁回該部分之請求，特此裁定。\n中華民國106年2月22日\n民事審查庭法官李怡諄\n正本係照原本作成。\n本裁定不得抗告。\n中華民國106年2月22日\n書記官葉明德'],
 [5,
  'PTDM,105,原選訴,1,20170612,1',
  '臺灣屏東地方法院刑事判決105年度原選訴字第1號\n105年度原選訴字第2號\n105年度原選訴字第3號\n公訴人臺灣屏東地方法院檢察署檢察官\n被告簡東明\n選任辯護人李衣婷律師\n林伯祥律師\n被告王榮儀\n選任辯護人鍾治漢律師\n被告馬昭明\n選任辯護人楊水柱律師\n被告李麗花\n蔡謨\n林國輝\n董忠\n唐建生\n杜文來\n高文生\n上七人共同指定辯護人吳曉維律師\n被告石英雄男55歲（民國00年00月0日生）\n身分證統一編號：Z000000000號\n住屏東縣○○鄉○○村○○00號之1\n董文巧男57歲（民國00年00月00日生）\n身分證統一編號：Z000000000號\n住屏東縣○○鄉○○村○○00○0號\n林華志男63歲（民國00年0月0日生）\n身分證統一編號：Z000000000號\n住屏東縣○○鄉○○村○○00○0號\n唐馨予女58歲（民國00年0月00日生）\n身分證統一編號：Z000000000號\n住屏東縣○○鄉○○村○○000號\n居屏東縣○○鄉○○村○○00○0號\n王育芳女60歲（民國00年0月00日生）\n身分證統一編號：Z000000000號\n住屏東縣○○鄉○○村○○00號\n董賢明男55歲（民國00年00月0日生）\n身分證統一編號：Z000000000號\n住屏東縣○○鄉○○村○○0○0號\n居屏東縣○○市○○街000巷00號\n陳英敏男48歲（民國00年0月0日生

In [45]:
merged_basic = basic_info_df[['Unnamed: 0', 'JID', 'JFULL']].copy()
print('basic_info_df Length:', len(merged_basic))
print('Last basic_info_df item:\n', merged_basic.iloc[-1])

# Merge basic_info and case_num_basic_info_case_type
matched_case_num_basic_info_case_type_list = [{'JID': JID, 'case_num': matched_groups[0], 'basic_info': matched_groups[1], 'case_type': matched_groups[2]} for matched_rule in case_num_basic_info_case_type_list[0] for input_text_index, JID, matched_groups, matched_groups_length in matched_rule]
matched_empty_case_num_basic_info_case_type_list = [{'JID': JID, 'case_num': matched_groups[0], 'basic_info': matched_groups[1], 'case_type': matched_groups[2]} for matched_rule in empty_case_num_basic_info_case_type_list[0] for input_text_index, JID, matched_groups, matched_groups_length in matched_rule]
matched_list = matched_case_num_basic_info_case_type_list + matched_empty_case_num_basic_info_case_type_list
matched_df = pd.DataFrame(matched_list)
print('matched_df Length:', len(matched_df))
print('matched_df Last item:', matched_df.iloc[-1])


merged = merged_basic.merge(matched_df, on='JID', how='left')
print('merged Length:', len(merged))
print('merged Last item:\n', merged.iloc[-1])
print('merged Valid Length:', len(merged.dropna()))

merged = merged.fillna('re error')
print('merged Length:', len(merged))
print('merged Last item:\n', merged.iloc[-1])
print('merged Valid Length:', len(merged.dropna()))
print('re error Length:', len(merged[merged['case_num']=='re error']))
output_csv_path = '/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/20240616_2017_2021_juds_包含有rn_filtered_basic_info_case_num_type_basic_info.csv'
# merged.to_csv(output_csv_path, encoding='utf-8-sig', index=False)

basic_info_df Length: 211813
Last basic_info_df item:
 Unnamed: 0                                              4135212
JID                                  TCDM,111,訴,2018,20230201,1
JFULL         臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
Name: 211812, dtype: object
matched_df Length: 211722
matched_df Last item: JID                               TPDM,111,審交附民移調,43,20220208,1
case_num                         臺灣臺北地方法院刑事判決\n111年度審交附民移調字第43號
basic_info    聲請人陳柏諺\n\n相對人陳守儀\n\n現於法務部○○○○○○○○\n上列當事人間就本院11...
case_type                           ２、聲請人願當庭撤回本院111年審交易字63號刑事案件
Name: 211721, dtype: object
merged Length: 211813
merged Last item:
 Unnamed: 0                                              4135212
JID                                  TCDM,111,訴,2018,20230201,1
JFULL         臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                            臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info             公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                       

In [46]:
merged = jud_read_csv(output_csv_path)


---Start reading csv file---
/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/20240616_2017_2021_juds_包含有rn_filtered_basic_info_case_num_type_basic_info.csv
20240616_2017_2021_juds_包含有rn_filtered_basic_info_case_num_type_basic_info.csv
Length: 211813 

Unnamed: 0                                              4135212
JID                                  TCDM,111,訴,2018,20230201,1
JFULL         臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                            臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info             公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                                        違反槍砲彈藥刀械管制條例案件
Name: 211812, dtype: object
---Finish reading csv file---



In [ ]:
# # Check if basic_info_df contains NaN
# print(len(basic_info_df[basic_info_df.isna().any(axis=1)]))
# # Check if basic_info_df['JID] contains NaN
# print(len(basic_info_df[basic_info_df.isna()['JID']]))

## 'court_type', 'jud_date', 'jud_url', 'syllabus'

In [65]:
# # ['UID', 'JID', 'jud_full', 'case_num', 'basic_info', 'case_type', 'court_type', 'jud_date', 'jud_url', 'syllabus']


# jud_date
def jud_re_jud_date_list(JIDs):
    print('---Start getting jud_dates---')
    jud_dates = [JID.split(',')[4] for JID in JIDs]
    print('jud_date Length:', len(jud_date))
    # Check if jud_date corrected
    jud_date_len = [len(JID.split(',')[4]) for JID in JIDs]
    print('If all jud_dates are length 8:', sorted(jud_date_len)[0]==sorted(jud_date_len)[-1] and sorted(jud_date_len)[0]==8)
    print('---Finish getting jud_dates---\n')
    return jud_dates

# court_type
def jud_re_court_type_list(JFULLs):
    court_types = []
    print('---Start getting court_types---')
    for JFULL in JFULLs:
        # court_pattern = r'(.+?法院)'
        court_pattern = r'(.+?)刑事判決'
        error_signs = ['\xa0', '\u3000', '\u30005', '\u3000111', '\u30004', '\u300011', '\u30003', '\u300024', '&nbsp;', '\u3000110', '\u300012', '\u300030', ' ', '\r']
        court_type = JFULL[:30]
        for sign in error_signs:
            court_type = court_type.replace(sign, '')
        try:
            court_type = re.search(court_pattern, court_type).group(1)
            # Filter with Traditional Chinese
            court_type = re.search(r'([\u4e00-\u9fff]+)', court_type).group(1)
        except:
            court_type = 're error, ' + court_type
        court_types.append(court_type)
    print('court_type Length:', len(court_types))
    errs = [tmp for tmp in court_types if tmp.startswith('re error')]
    print('re error Length:', len(errs))
    print('Unique court types: ', len(list(set(court_types))))
    print('---Finish getting court_types---\n')
    return court_types
def jud_re_jud_url_list(JIDs):
    print('---Start getting jud_urls---')
    judical_yuan_url_prefix = 'https://judgment.judicial.gov.tw/FJUD/data.aspx?ty=JD&id={}'
    jud_urls = [judical_yuan_url_prefix.format(JID) for JID in JIDs]
    print('jud_url Length:', len(jud_urls))
    print('---Finish getting jud_urls---\n')
    return jud_urls

def jud_re_syllabus_list(JFULLs):
    syllabus_list = []
    print('---Start getting syllabus---')

    for JFULL in tqdm(JFULLs):
        JFULL = JFULL.replace(" ","").replace(r"\n","").replace("\r","").replace("\t","")
        # JFULL=JFULL.replace(" ","").replace("\r","").replace("\t","")
        try:
            matched_text = re.search("(主文)(.*?)(犯罪事實|事實及理由|事實|理由|犯罪)",JFULL,flags=re.S)
            syllabus = matched_text.group(0).split("主文")[1]
            symbols = ["事實及","事實","理由","犯罪", '\n']
            for symbol in symbols:
                syllabus = syllabus.replace(symbol,"")
            syllabus_list.append(syllabus)
        except:
            syllabus_list.append('re error')
    print('syllabus Length:', len(syllabus_list))
    errs = [tmp for tmp in syllabus_list if tmp.startswith('re error')]
    print('re error Length:', len(errs))
    print('---Finish getting syllabus---\n')
    return syllabus_list
jud_date = jud_re_jud_date_list(basic_info_df['JID'])
court_type = jud_re_court_type_list(basic_info_df['JFULL'])
jud_url = jud_re_jud_url_list(basic_info_df['JID'])
syllabus = jud_re_syllabus_list(basic_info_df['JFULL'])

basic_info_df['court_type'] = court_type
basic_info_df['jud_date'] = jud_date
basic_info_df['jud_url'] = jud_url
basic_info_df['syllabus'] = syllabus
print('No nan in basic_info_df:', len(basic_info_df[basic_info_df.isna().any(axis=1)])==0)
basic_info_df.head(3)



---Start getting jud_dates---
jud_date Length: 211813
If all jud_dates are length 8: True
---Finish getting jud_dates---

---Start getting court_types---
court_type Length: 211813
re error Length: 39
Unique court types:  113
---Finish getting court_types---

---Start getting jud_urls---
jud_url Length: 211813
---Finish getting jud_urls---

---Start getting syllabus---


100%|██████████| 211813/211813 [00:16<00:00, 13221.74it/s]


syllabus Length: 211813
re error Length: 47
---Finish getting syllabus---

No nan in basic_info_df: True


,Unnamed: 0,JID,JFULL,case_num,basic_info,case_type,court_type,jud_date,jud_url,syllabus
0,82,"TPHM,105,上訴,2019,20160819,1",臺灣高等法院刑事判決 105年度上訴字第2019號\r\n上 訴 人\r\n即...,臺灣高等法院刑事判決105年度上訴字第2019號,上訴人\n即被告郭承滄\n,違反毒品危害防制條例案件,臺灣高等法院,20160819,https://judgment.judicial.gov.tw/FJUD/data.asp...,上訴駁回。
1,244,"TPHM,105,上訴,2018,20161005,1",臺灣高等法院刑事判決 105年度上訴字第2018號\r\n上 訴 人\r\n即...,臺灣高等法院刑事判決105年度上訴字第2018號,上訴人\n即被告姚鵬\n選任辯護人羅美鈴律師\n,違反毒品危害防制條例案件,臺灣高等法院,20161005,https://judgment.judicial.gov.tw/FJUD/data.asp...,原判決撤銷。姚鵬犯如附表
2,245,"TPHM,105,上訴,2021,20161013,1",臺灣高等法院刑事判決 105年度上訴字第2021號\r\n上 訴 人\r\n即...,臺灣高等法院刑事判決105年度上訴字第2021號,上訴人\n即被告施明峰\n選任辯護人林傳欽律師\n,違反槍砲彈藥刀械管制條例等案件,臺灣高等法院,20161013,https://judgment.judicial.gov.tw/FJUD/data.asp...,原判決關於寄藏改造手槍部分撤銷。施明峰犯未經許可，寄藏可發射子彈具有殺傷力之改造手槍，累犯，...


In [75]:
# # 依照日期排序檔案
basic_info_df.sort_values('jud_date')
print('First 50 dates:\n', basic_info_df['jud_date'][:50])
print('-----')
print('Last 50 dates:\n',basic_info_df['jud_date'][-50:])
basic_info_df.head(5)

First 50 dates:
 0     20160819
1     20161005
2     20161013
3     20161111
4     20161111
5     20170112
6     20170118
7     20170119
8     20170125
9     20170119
10    20170105
11    20170105
12    20170105
13    20170112
14    20170112
15    20170119
16    20170118
17    20170105
18    20170105
19    20170119
20    20170125
21    20170112
22    20170119
23    20170105
24    20170104
25    20170105
26    20170105
27    20170112
28    20170105
29    20170104
30    20170112
31    20170112
32    20170104
33    20170105
34    20170112
35    20170105
36    20170105
37    20170111
38    20170112
39    20170112
40    20170118
41    20170125
42    20170105
43    20170125
44    20170105
45    20170119
46    20170118
47    20170119
48    20170104
49    20170111
Name: jud_date, dtype: int64
-----
Last 50 dates:
 211763    20220217
211764    20220217
211765    20220214
211766    20220215
211767    20220208
211768    20220210
211769    20220210
211770    20220207
211771    20220216
211772    2

,2017_2021_juds_包含有rn_remove_duplicated_id,JID,JFULL,case_num,basic_info,case_type,court_type,jud_date,jud_url,syllabus
0,82,"TPHM,105,上訴,2019,20160819,1",臺灣高等法院刑事判決 105年度上訴字第2019號\r\n上 訴 人\r\n即...,臺灣高等法院刑事判決105年度上訴字第2019號,上訴人\n即被告郭承滄\n,違反毒品危害防制條例案件,臺灣高等法院,20160819,https://judgment.judicial.gov.tw/FJUD/data.asp...,上訴駁回。
1,244,"TPHM,105,上訴,2018,20161005,1",臺灣高等法院刑事判決 105年度上訴字第2018號\r\n上 訴 人\r\n即...,臺灣高等法院刑事判決105年度上訴字第2018號,上訴人\n即被告姚鵬\n選任辯護人羅美鈴律師\n,違反毒品危害防制條例案件,臺灣高等法院,20161005,https://judgment.judicial.gov.tw/FJUD/data.asp...,原判決撤銷。姚鵬犯如附表
2,245,"TPHM,105,上訴,2021,20161013,1",臺灣高等法院刑事判決 105年度上訴字第2021號\r\n上 訴 人\r\n即...,臺灣高等法院刑事判決105年度上訴字第2021號,上訴人\n即被告施明峰\n選任辯護人林傳欽律師\n,違反槍砲彈藥刀械管制條例等案件,臺灣高等法院,20161013,https://judgment.judicial.gov.tw/FJUD/data.asp...,原判決關於寄藏改造手槍部分撤銷。施明峰犯未經許可，寄藏可發射子彈具有殺傷力之改造手槍，累犯，...
3,258,"PCDM,105,審訴,2019,20161111,1",臺灣新北地方法院刑事判決 105年度審訴字第2019號\r\n公 訴 人 臺灣新北...,臺灣新北地方法院刑事判決105年度審訴字第2019號,公訴人臺灣新北地方法院檢察署檢察官\n被告張正羣\n,偽造文書等案件,臺灣新北地方法院,20161111,https://judgment.judicial.gov.tw/FJUD/data.asp...,本件管轄錯誤，移送於臺灣臺中地方法院。
4,297,"TPHM,105,上訴,2017,20161111,1",臺灣高等法院刑事判決 105年度上訴字第2017號\r\n上 訴 人 陳忠耀\...,臺灣高等法院刑事判決105年度上訴字第2017號,上訴人陳忠耀\n即被告\n選任辯護人徐國楨律師\n,業務過失致死案件,臺灣高等法院,20161111,https://judgment.judicial.gov.tw/FJUD/data.asp...,上訴駁回。


In [76]:
# # 輸出檔案
# output_csv_path = '/workspace/data/CDB-2017-2021/5. Final RE files/20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info.csv'
# basic_info_df.to_csv(output_csv_path, encoding='utf-8-sig', index=False)
# 檢查檔案
basic_info_df = jud_read_csv(output_csv_path)

# # 檢查df中所有cell包含re error的資料
# print(basic_info_df[basic_info_df.apply(lambda x: x.astype(str).str.contains('re error')).any(axis=1)])

# # 刪除欄位, 欄位改名
# basic_info_df = basic_info_df.drop(columns=['Unnamed: 0.1'])
# basic_info_df = basic_info_df.rename(columns={'2017_2021_juds_包含有rn_remove_duplicated': '2017_2021_juds_包含有rn_remove_duplicated_id'})
# basic_info_df.iloc[-1]

---Start reading csv file---
/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info.csv
20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info.csv
Length: 211813 

2017_2021_juds_包含有rn_remove_duplicated_id                                              4135212
JID                                                                 TCDM,111,訴,2018,20230201,1
JFULL                                        臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                                                           臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info                                            公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                                                                       違反槍砲彈藥刀械管制條例案件
court_type                                                                            臺灣臺中地方法院
jud_date                                                                              20230201
jud

# Merge old basic_info and new basic_info

## Original new and old basic_info

In [6]:
# # Before merged
new_basic_csv_path = '/workspace/data/CDB-2017-2021/5. Final RE files/20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info.csv'
# new_basic_info_df = jud_read_csv(new_basic_csv_path)

old_basic_csv_path = '/workspace/data/CDB_20240304_110juds/20240320_final_merged/20240320_110_basic_info.csv'
# old_basic_info_df = jud_read_csv(old_basic_csv_path)



---Start reading csv file---
Length: 211813 

2017_2021_juds_包含有rn_remove_duplicated_id                                              4135212
JID                                                                 TCDM,111,訴,2018,20230201,1
JFULL                                        臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                                                           臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info                                            公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                                                                       違反槍砲彈藥刀械管制條例案件
court_type                                                                            臺灣臺中地方法院
jud_date                                                                              20230201
jud_url                                      https://judgment.judicial.gov.tw/FJUD/data.asp...
syllabus                                     許綜仁犯非法持有非制式衝鋒槍罪，處有期徒刑伍年捌月，併科罰金新臺幣拾萬元，罰金如易服勞役，以...
Name

## merge function

In [15]:
old_category_path = {'fee': '/workspace/data/CDB_20240304_110juds/20240320_final_merged/20240322_110_category_fee.csv',
                'opinion': '/workspace/data/CDB_20240304_110juds/20240320_final_merged/20240322_110_category_opinion.csv',
                'sub': '/workspace/data/CDB_20240304_110juds/20240320_final_merged/20240322_110_category_sub.csv'
                }
new_category_path = {'fee': '/workspace/data/CDB-2017-2021/5. Final RE files/2017_2021判決書_無簡_paragraph_20240331_predicted_fee_target.csv',
                'opinion': '/workspace/data/CDB-2017-2021/5. Final RE files/2017_2021判決書_無簡_sentence_20240331_predicted_opinions_target.csv',
                'sub': '/workspace/data/CDB-2017-2021/5. Final RE files/2017_2021判決書_無簡_paragraph_20240331_predicted_sub_target.csv' 
                }
def jud_remove_duplicate_and_merge_df(new_csv_path, old_csv_path, common_columns, save_file=False):
    print('---Start removing duplicate data---')
    new_df = jud_read_csv(new_csv_path, '\n')
    old_df = jud_read_csv(old_csv_path, '\n')
    not_overlapped = new_df[~new_df['JID'].isin(old_df['JID'])]
    not_overlapped = jud_read_csv(not_overlapped)
    # output_file_name = os.path.basename(new_csv_path).split('.')[0] + '_not_overlapped.csv'
    output_file_name = os.path.basename(new_csv_path).split('.')[0] + '_not_overlapped_merged.csv'
    output_not_overlapped_path = os.path.dirname(new_csv_path) + '/' + output_file_name
    
    # Merged old and new 
    # prediction
    # ['JID', 'sentence']
    not_overlapped = not_overlapped[common_columns].copy()

    # # basic_info
    # old_df = old_df.rename(columns={'jud_full': 'JFULL'})
    old_df = old_df[common_columns].copy()
    merged_df = old_df.merge(not_overlapped, how='outer')
    if save_file:
        print('File saved: ',output_not_overlapped_path)
        # not_overlapped.to_csv(output_not_overlapped_path, index=False)
        merged_df.to_csv(output_not_overlapped_path, index=False)

        print('Checking saved file')
        merged_df = jud_read_csv(output_not_overlapped_path, '\n')
    print('---Finish removing duplicate data---\n')

    return merged_df


## Basic_info merged

In [9]:

new_basic_csv_path = '/workspace/data/CDB-2017-2021/5. Final RE files/20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info.csv'
old_basic_csv_path = '/workspace/data/CDB_20240304_110juds/20240320_final_merged/20240320_110_basic_info.csv'
basic_info_columns = ['JID', 'JFULL', 'case_num', 'basic_info', 'case_type', 'court_type', 'jud_date', 'jud_url', 'syllabus']

merged_basic_info_df = jud_remove_duplicate_and_merge_df(new_basic_csv_path, old_basic_csv_path, basic_info_columns, True)

---Start removing duplicate data---
---Start reading csv file---
/workspace/data/CDB-2017-2021/5. Final RE files/20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info.csv
20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info.csv
Length: 211813 

2017_2021_juds_包含有rn_remove_duplicated_id                                              4135212
JID                                                                 TCDM,111,訴,2018,20230201,1
JFULL                                        臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                                                           臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info                                            公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                                                                       違反槍砲彈藥刀械管制條例案件
court_type                                                                            臺灣臺中地方法院
jud_date                                                                           

In [17]:
merged_basic_info_df = merged_basic_info_df.sort_values('jud_date')
# # 依照日期排序檔案
merged_basic_info_df.sort_values('jud_date')
print('First 10 dates:\n', merged_basic_info_df['jud_date'][:10])
print('-----')
print('Last 10 dates:\n',merged_basic_info_df['jud_date'][-10:])
merged_basic_info_df.head(3)
# merged_basic_info_df.to_csv('/workspace/data/CDB-2017-2021/5. Final RE files/20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info_not_overlapped_merged.csv', encoding='utf-8-sig', index=False)


First 10 dates:
 169918    20160819
169917    20161005
169920    20161013
169916    20161111
63844     20161111
109743    20170102
127905    20170103
127640    20170103
170167    20170103
127607    20170103
Name: jud_date, dtype: int64
-----
Last 10 dates:
 194473    20220830
194475    20220831
139876    20221103
139865    20221116
139873    20221124
127502    20221227
127499    20221228
127503    20230112
127498    20230116
127497    20230201
Name: jud_date, dtype: int64


In [2]:
merged_basic_info_df = jud_read_csv('/workspace/data/CDB-2017-2021/5. Final RE files/20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info_not_overlapped_merged.csv')


---Start reading csv file---
/workspace/data/CDB-2017-2021/5. Final RE files/20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info_not_overlapped_merged.csv
20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info_not_overlapped_merged.csv
Length: 246600 

JID                                  TCDM,111,訴,2018,20230201,1
JFULL         臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                            臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info             公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                                        違反槍砲彈藥刀械管制條例案件
court_type                                             臺灣臺中地方法院
jud_date                                               20230201
jud_url       https://judgment.judicial.gov.tw/FJUD/data.asp...
syllabus      許綜仁犯非法持有非制式衝鋒槍罪，處有期徒刑伍年捌月，併科罰金新臺幣拾萬元，罰金如易服勞役，以...
Name: 246599, dtype: object
---Finish reading csv file---



## Category merged

In [6]:
merged_sub = jud_remove_duplicate_and_merge_df(new_category_path['sub'], old_category_path['sub'], ['JID', 'sentence'], True)


---Start removing duplicate data---
---Start reading csv file---
/workspace/data/CDB-2017-2021/5. Final RE files/2017_2021判決書_無簡_paragraph_20240331_predicted_sub_target.csv
2017_2021判決書_無簡_paragraph_20240331_predicted_sub_target.csv
Length: 1775704 

Unnamed: 0                                              3638730
JID                                  TCDM,111,訴,2018,20230201,1
sentence      ㈢至其餘扣案之被告持用手機1支、證人黃富典持用手機1支及燈具支架2個、燈具電線開關1條等物參...
prediction                                                   目標
Name: 1775703, dtype: object
---Finish reading csv file---

---Start reading csv file---
/workspace/data/CDB_20240304_110juds/20240320_final_merged/20240322_110_category_sub.csv
20240322_110_category_sub.csv
Length: 495427 

EID                                                    495426
JID                                FYEM,110,豐交易,23,20211224,1
sentence    三、本件被告李啟鑫經檢察官以渉犯刑法第284條前段之過失傷害罪聲請簡易判決處刑，依同法第28...
type                                                       目標
UID                 

In [10]:
print('Merged sub Length:', len(merged_sub))
print('All sub in merged_basic_info:', len(merged_sub[merged_sub['JID'].isin(merged_basic_info_df['JID'])])==len(merged_sub))

Merged sub Length: 1918493
All sub in merged_basic_info: True


In [16]:
merged_opinion = jud_remove_duplicate_and_merge_df(new_category_path['opinion'], old_category_path['opinion'], ['JID', 'sentence'], True)

---Start removing duplicate data---
---Start reading csv file---
/workspace/data/CDB-2017-2021/5. Final RE files/2017_2021判決書_無簡_sentence_20240331_predicted_opinions_target.csv
2017_2021判決書_無簡_sentence_20240331_predicted_opinions_target.csv
Length: 1156480 

Unnamed: 0                                              8347521
JID                                  TCDM,111,訴,2018,20230201,1
sentence      若同時持有\r、寄藏二不相同種類之客體（如同時寄藏手槍及子彈），則\r為一行為觸犯數罪名之想...
prediction                                                   目標
Name: 1156479, dtype: object
---Finish reading csv file---

---Start reading csv file---
/workspace/data/CDB_20240304_110juds/20240320_final_merged/20240322_110_category_opinion.csv
20240322_110_category_opinion.csv
Length: 364544 

EID                                                    364543
JID                                 KSDM,110,金訴緝,4,20211230,1
sentence    又依被告本案所為犯罪行為及實質侵害法益之質與量，如以實質累加之方式定其所犯如附表所示之罪應執...
type                                                       目標
UID 

In [17]:
print('Merged opinion Length:', len(merged_opinion))
print('All opinion in merged_basic_info:', len(merged_opinion[merged_opinion['JID'].isin(merged_basic_info_df['JID'])])==len(merged_opinion))

Merged opinion Length: 1276442
All opinion in merged_basic_info: True


In [18]:
merged_fee = jud_remove_duplicate_and_merge_df(new_category_path['fee'], old_category_path['fee'], ['JID', 'sentence'], True)
print('Merged fee Length:', len(merged_fee))
print('All fee in merged_basic_info:', len(merged_fee[merged_fee['JID'].isin(merged_basic_info_df['JID'])])==len(merged_fee))

---Start removing duplicate data---
---Start reading csv file---
/workspace/data/CDB-2017-2021/5. Final RE files/2017_2021判決書_無簡_paragraph_20240331_predicted_fee_target.csv
2017_2021判決書_無簡_paragraph_20240331_predicted_fee_target.csv
Length: 127513 

Unnamed: 0                                              3638711
JID                                 TCDM,111,金訴,2018,20230112,1
sentence      3.再查，被告於警詢、檢察事務官詢問及本院審理時供認：我一開始看到該投資廣告時有覺得不合理，...
prediction                                                   目標
Name: 127512, dtype: object
---Finish reading csv file---

---Start reading csv file---
/workspace/data/CDB_20240304_110juds/20240320_final_merged/20240322_110_category_fee.csv
20240322_110_category_fee.csv
Length: 35497 

EID                                                     35496
JID                                 KSDM,110,金訴,93,20211230,1
sentence    ⑷本院衡酌近年來各式各樣之詐欺取財犯罪類型層出不窮，詐欺集團透過網路刊登不實訊息，致被害人與...
type                                                       目標
UID                    

# Check all merged information

In [4]:
basic_info_path = '/workspace/data/CDB-2017-2021/6. Merged df/20240617_2017_2021_juds_basic_info.csv'
sub_path = '/workspace/data/CDB-2017-2021/6. Merged df/20240617_2017_2021判決書_無簡_paragraph_sub.csv'
opinion_path = '/workspace/data/CDB-2017-2021/6. Merged df/20240617_2017_2021判決書_無簡_sentence_opinion.csv'
fee_path = '/workspace/data/CDB-2017-2021/6. Merged df/20240617_2017_2021判決書_無簡_paragraph_fee.csv'
basic_info = jud_read_csv(basic_info_path, '\n')
sub = jud_read_csv(sub_path, '\n')
opinion = jud_read_csv(opinion_path, '\n')
fee = jud_read_csv(fee_path, '\n')


---Start reading csv file---
/workspace/data/CDB-2017-2021/6. Merged df/20240617_2017_2021_juds_basic_info.csv
20240617_2017_2021_juds_basic_info.csv
Length: 246600 

UID                                                      246599
JID                                  TCDM,111,訴,2018,20230201,1
JFULL         臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                            臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info             公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                                        違反槍砲彈藥刀械管制條例案件
court_type                                             臺灣臺中地方法院
jud_date                                               20230201
jud_url       https://judgment.judicial.gov.tw/FJUD/data.asp...
syllabus      許綜仁犯非法持有非制式衝鋒槍罪，處有期徒刑伍年捌月，併科罰金新臺幣拾萬元，罰金如易服勞役，以...
Name: 246599, dtype: object
---Finish reading csv file---

---Start reading csv file---
/workspace/data/CDB-2017-2021/6. Merged df/20240617_2017_2021判決書_無簡_paragraph_sub.csv
20240617_2017_2021判

# Create UID to all csv.
UID is a substitution of JID

In [15]:
def jud_assign_uid_to_prediction(basic_info, prediction):
    output_df = prediction.copy()
    print('Prediction Length:', len(output_df))
    output_df = output_df.merge(basic_info[['JID', 'UID']], on='JID', how='left')
    print('Prediction Length:', len(output_df))
    print('Last data of prediction:\n', output_df.iloc[-1])
    return output_df
basic_info.insert(0, 'UID', range(len(basic_info)), False)
print(basic_info.iloc[-1])
print()
sub = jud_assign_uid_to_prediction(basic_info, sub)
opinion = jud_assign_uid_to_prediction(basic_info, opinion)
fee = jud_assign_uid_to_prediction(basic_info, fee)

UID                                                      246599
JID                                  TCDM,111,訴,2018,20230201,1
JFULL         臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                            臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info             公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                                        違反槍砲彈藥刀械管制條例案件
court_type                                             臺灣臺中地方法院
jud_date                                               20230201
jud_url       https://judgment.judicial.gov.tw/FJUD/data.asp...
syllabus      許綜仁犯非法持有非制式衝鋒槍罪，處有期徒刑伍年捌月，併科罰金新臺幣拾萬元，罰金如易服勞役，以...
Name: 246599, dtype: object

Prediction Length: 1918493
Prediction Length: 1918493
Last data of prediction:
 JID                                  ULDM,111,金訴,2,20220214,1
sentence    貳、程序部分被告乙○○所犯之罪，係死刑、無期徒刑、最輕本刑為3年以上有期徒刑以外之罪，亦非高...
UID                                                    245545
Name: 1918492, dtype: object
Prediction Length: 1276442
Predictio

In [17]:
sub[-10:]

,JID,sentence,UID
1918483,"ULDM,111,簡,195,20210930,1",㈤、按犯罪之情狀顯可憫恕，認科以最低刑度仍嫌過重者，得酌量減輕其刑，刑法第59條定有明文，所...,222639
1918484,"ULDM,111,簡,195,20210930,1",三、法官考量刑度的理由除了前開審酌刑法第59條時所考慮的事由外，主要還是在犯罪現場中，被告丙...,222639
1918485,"ULDM,111,簡,195,20210930,1",二、論罪科刑部分㈠、按刑法第150條聚眾施強暴脅迫罪，原規定之構成要件「公然聚眾」部分，於1...,222639
1918486,"ULDM,111,簡,195,20210930,1",查被告乙○○、丙○○及2位少年於聚集施以強暴過程中，主觀上均已有對他人造成恐懼或危害之認識或...,222639
1918487,"ULDM,111,金訴,2,20220214,1",㈡、經查，被告因對附表「被害人」欄所示之人加以訛騙，致其等陷於錯誤，而分別匯款如附表「匯款金...,245545
1918488,"ULDM,111,金訴,2,20220214,1",三、被告前因竊盜、偽造文書、詐欺、違反兒童及少年性交易防制條例等案件，經法院分別判處罪刑確定...,245545
1918489,"ULDM,111,金訴,2,20220214,1",二、核被告就附表編號1至9所為，均係犯刑法第339條第1項詐欺取財罪及違反洗錢防制法第2條第...,245545
1918490,"ULDM,111,金訴,2,20220214,1",參、認定犯罪事實所憑之證據及理由上揭犯罪事實，均業經被告坦承不諱，並有下列證據可佐：證人即告...,245545
1918491,"ULDM,111,金訴,2,20220214,1",肆、論罪科刑一、洗錢防制法所稱之「洗錢」行為，依第2條之規定，係指：一、意圖掩飾或隱匿特定犯...,245545
1918492,"ULDM,111,金訴,2,20220214,1",貳、程序部分被告乙○○所犯之罪，係死刑、無期徒刑、最輕本刑為3年以上有期徒刑以外之罪，亦非高...,245545


In [22]:
# Save df
# sub.to_csv(sub_path, encoding='utf-8-sig', index=False)
# opinion.to_csv(opinion_path, encoding='utf-8-sig', index=False)
# fee.to_csv(fee_path, encoding='utf-8-sig', index=False)
basic_info.to_csv(basic_info_path, encoding='utf-8-sig', index=False)

basic_info = jud_read_csv(basic_info_path, '\n')
sub = jud_read_csv(sub_path, '\n')
opinion = jud_read_csv(opinion_path, '\n')
fee = jud_read_csv(fee_path, '\n')


---Start reading csv file---
/workspace/data/CDB-2017-2021/6. Merged df/20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info_not_overlapped_merged.csv
20240617_2017_2021_juds_包含有rn_filtered_re_all_basic_info_not_overlapped_merged.csv
Length: 246600 

UID                                                      246599
JID                                  TCDM,111,訴,2018,20230201,1
JFULL         臺灣臺中地方法院刑事判決\r\n111年度訴字第2018號\r\n公  訴  人  臺灣臺中...
case_num                            臺灣臺中地方法院刑事判決\n111年度訴字第2018號
basic_info             公訴人臺灣臺中地方檢察署檢察官\n被告許綜仁\n\n\n選任辯護人曹合一律師\n
case_type                                        違反槍砲彈藥刀械管制條例案件
court_type                                             臺灣臺中地方法院
jud_date                                               20230201
jud_url       https://judgment.judicial.gov.tw/FJUD/data.asp...
syllabus      許綜仁犯非法持有非制式衝鋒槍罪，處有期徒刑伍年捌月，併科罰金新臺幣拾萬元，罰金如易服勞役，以...
Name: 246599, dtype: object
---Finish reading csv file---

---Start reading csv file---
/workspace/data/C

# 

In [21]:
basic_info.iloc[-1]
len(basic_info)

246600

# Test

In [22]:
def get_keywords_df(query_text, target_df, target_column = 'sentence'):
    if query_text=="":
        return target_df
    tmp_df = target_df
    tmp_df = tmp_df.fillna("無資料")
    for keyword in query_text.split(' '):
        if keyword!="":
            tmp_df = tmp_df[tmp_df[target_column].str.contains(keyword)]
    tmp_df = tmp_df.drop_duplicates()
    return tmp_df

# get_keywords_df('50553', basic_info, 'UID')
# print(basic_info[basic_info['UID']==50553]['JFULL'].iloc[0])
print(basic_info[basic_info['UID']==50553].iloc[0])
# print(basic_info[basic_info['UID']==50806]['JFULL'].iloc[0])
print(basic_info[basic_info['UID']==50806].iloc[0])

UID                                                       50553
JID                                  TCDM,106,訴,1827,20180430,2
JFULL         臺灣臺中地方法院刑事判決　　　　　　　105年度訴字第621號\r\n　　　　　　　　　　　...
case_num                               臺灣臺中地方法院刑事判決105年度訴字第621號
basic_info    105年度訴字第974號\n105年度訴字第1181號\n105年度訴字第1457號\n10...
case_type                                                  詐欺案件
court_type                                             臺灣臺中地方法院
jud_date                                               20180430
jud_url       https://judgment.judicial.gov.tw/FJUD/data.asp...
syllabus      林詠欽、高健祐犯附表三各編號所示之罪，各處附表三各編號所示之主刑及沒收。林詠欽應執行有期徒刑...
Name: 50553, dtype: object
UID                                                       50806
JID                                   TCDM,105,訴,621,20180430,2
JFULL         臺灣臺中地方法院刑事判決　　　　　　　105年度訴字第621號\r\n　　　　　　　　　　　...
case_num                               臺灣臺中地方法院刑事判決105年度訴字第621號
basic_info    105年度訴字第974號\n105年度訴字第1181號\n105年度訴字第1457號\n10...
case_type    

In [23]:
flitered = pd.read_csv('/workspace/data/CDB-2017-2021/4. Filtered basic_info and predictions/20240616_2017_2021_juds_包含有rn_filtered_basic_info.csv')

In [29]:
text = 'TCDM,106,訴,1827,20180430,2'
flitered[flitered['JID']==text].iloc[0]
print(len(flitered[flitered['JID']==text].iloc[0]['JFULL']))

print(flitered[flitered['JID']==text].iloc[0]['JFULL'])

162544
臺灣臺中地方法院刑事判決　　　　　　　105年度訴字第621號
　　　　　　　　　　　　　　　　　　　105年度訴字第974號
　　　　　　　　　　　　　　　　　　 105年度訴字第1181號
　　　　　　　　　　　　　　　　　　 105年度訴字第1457號
　　　　　　　　　　　　　　　　　　 106年度訴字第1827號
公　訴　人　臺灣臺中地方法院檢察署檢察官
被　　　告　林詠欽
選任辯護人　吳佳原律師
被　　　告　高健祐
選任辯護人　陳如梅律師
被　　　告　林佑
選任辯護人　周復興律師
被　　　告　范博瑋
選任辯護人　薛逢逸律師
被　　　告　麥詩佳
上列被告因詐欺案件，經檢察官提起公訴（105 年度偵字第3741
、11250 、12452 號）、移送併案審理（105 年度偵字第00000
號）及追加起訴（105 年度偵字第7498、13888 、14509 、2619
7 號、106 年度偵字第13434 號），本院判決如下：
    主    文
林詠欽、高健祐犯附表三各編號所示之罪，各處附表三各編號所
示之主刑及沒收。林詠欽應執行有期徒刑柒年捌月，高健祐應執
行有期徒刑柒年肆月。
范博瑋犯附表四各編號所示之罪，處附表四各編號所示之主刑及
沒收。應執行有期徒刑參年貳月。
林佑犯附表五各編號所示之罪，處附表五各編號所示之主刑及沒
收。應執行有期徒刑肆年貳月。
麥詩佳犯附表六各編號所示之罪，處附表六各編號所示之主刑及
沒收。應執行有期徒刑肆年貳月。
范博瑋、林佑、麥詩佳其餘被訴加重詐欺部分，均無罪。
    犯罪事實
一、林詠欽（綽號「阿德」、「阿der 」）於民國103 年10月間
    ，為獲取不詳之報酬及分紅，加入真實姓名年籍不詳自稱「
    陳桂秋」之成年女子等人所組成之詐欺集團（下稱「陳桂秋
    」詐騙集團」，負責蒐集帳戶以供被害人匯入款項，並親自
    擔任提款車手及通知其他車手提領款項之俗稱「車手頭」工
    作，將車手成員領出之款項扣除不詳之報酬後，經由匯款交
    付予「陳桂秋」，且於103 年12月間，以代繳房租、水電費
    、管理費等做為報酬，邀集與其一同租屋在臺中市○○區○
    ○路○段000 ○00號3 樓之市政LV大樓房屋（下稱文心路租
    處）之大學學長范博瑋（綽號「屁屁」）擔任提款車手工作

In [28]:
text = 'TCDM,105,訴,621,20180430,2'
flitered[flitered['JID']==text].iloc[0]
print(len(flitered[flitered['JID']==text].iloc[0]['JFULL']))
print(flitered[flitered['JID']==text].iloc[0]['JFULL'])

162544
臺灣臺中地方法院刑事判決　　　　　　　105年度訴字第621號
　　　　　　　　　　　　　　　　　　　105年度訴字第974號
　　　　　　　　　　　　　　　　　　 105年度訴字第1181號
　　　　　　　　　　　　　　　　　　 105年度訴字第1457號
　　　　　　　　　　　　　　　　　　 106年度訴字第1827號
公　訴　人　臺灣臺中地方法院檢察署檢察官
被　　　告　林詠欽
選任辯護人　吳佳原律師
被　　　告　高健祐
選任辯護人　陳如梅律師
被　　　告　林佑
選任辯護人　周復興律師
被　　　告　范博瑋
選任辯護人　薛逢逸律師
被　　　告　麥詩佳
上列被告因詐欺案件，經檢察官提起公訴（105 年度偵字第3741
、11250 、12452 號）、移送併案審理（105 年度偵字第00000
號）及追加起訴（105 年度偵字第7498、13888 、14509 、2619
7 號、106 年度偵字第13434 號），本院判決如下：
    主    文
林詠欽、高健祐犯附表三各編號所示之罪，各處附表三各編號所
示之主刑及沒收。林詠欽應執行有期徒刑柒年捌月，高健祐應執
行有期徒刑柒年肆月。
范博瑋犯附表四各編號所示之罪，處附表四各編號所示之主刑及
沒收。應執行有期徒刑參年貳月。
林佑犯附表五各編號所示之罪，處附表五各編號所示之主刑及沒
收。應執行有期徒刑肆年貳月。
麥詩佳犯附表六各編號所示之罪，處附表六各編號所示之主刑及
沒收。應執行有期徒刑肆年貳月。
范博瑋、林佑、麥詩佳其餘被訴加重詐欺部分，均無罪。
    犯罪事實
一、林詠欽（綽號「阿德」、「阿der 」）於民國103 年10月間
    ，為獲取不詳之報酬及分紅，加入真實姓名年籍不詳自稱「
    陳桂秋」之成年女子等人所組成之詐欺集團（下稱「陳桂秋
    」詐騙集團」，負責蒐集帳戶以供被害人匯入款項，並親自
    擔任提款車手及通知其他車手提領款項之俗稱「車手頭」工
    作，將車手成員領出之款項扣除不詳之報酬後，經由匯款交
    付予「陳桂秋」，且於103 年12月間，以代繳房租、水電費
    、管理費等做為報酬，邀集與其一同租屋在臺中市○○區○
    ○路○段000 ○00號3 樓之市政LV大樓房屋（下稱文心路租
    處）之大學學長范博瑋（綽號「屁屁」）擔任提款車手工作